In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Exploration des émissions de co2

In [5]:
emissions = pd.read_csv('../data/raw/emission-co2-perimetre-complet.csv',sep=';')
print(emissions.shape)
emissions.head()

(119, 11)


,Transporteur,Origine,Origine_uic,Destination,Destination_uic,Distance entre les gares,Train - Empreinte carbone (kgCO2e),Autocar longue distance - Empreinte carbone (kgCO2e),Avion - Empreinte carbone (kgCO2e),"Voiture électrique (2,2 pers.) - Empreinte carbone (kgCO2e)","Voiture thermique (2,2 pers.) - Empreinte carbone (kgCO2e)"
0,TGV,Paris Est,87113001,Metz,87192039,352.0,1.0208,11.4540,NaN,15.604,35.7896
1,TGV,Aeroport CDG2 TGV Roissy,87271494,Lille Europe,87223263,203.0,0.5887,6.9345,NaN,9.447,21.6678
2,International,Paris Nord,88194060,Bruxelles N Aero,88194060,314.0,2.4178,10.7640,37.9745,14.664,33.6336
3,International,Paris Gare de Lyon,87686006,Geneve,85010082,503.0,1.7102,18.6645,59.0396,25.427,58.3198
4,TGV,Marne la Vallee Chessy,87111849,Avignon TGV,87318964,646.0,1.8734,24.1845,NaN,32.947,75.5678


In [6]:
def rename_columns(emissions_df):
    emissions_df.rename(columns={
        'Distance entre les gares':'Distance', 
        "Train - Empreinte carbone (kgCO2e)":"Train", 
        "Autocar longue distance - Empreinte carbone (kgCO2e)": "Autocar", 
        "Avion - Empreinte carbone (kgCO2e)":"Avion", 
        "Voiture électrique (2,2 pers.) - Empreinte carbone (kgCO2e)":"Voiture électrique", 
        "Voiture thermique (2,2 pers.) - Empreinte carbone (kgCO2e)":"Voiture thermique"
    }, inplace=True)
    return emissions_df

In [7]:
emissions = rename_columns(emissions)
emissions.head()

,Transporteur,Origine,Origine_uic,Destination,Destination_uic,Distance,Train,Autocar,Avion,Voiture électrique,Voiture thermique
0,TGV,Paris Est,87113001,Metz,87192039,352.0,1.0208,11.4540,NaN,15.604,35.7896
1,TGV,Aeroport CDG2 TGV Roissy,87271494,Lille Europe,87223263,203.0,0.5887,6.9345,NaN,9.447,21.6678
2,International,Paris Nord,88194060,Bruxelles N Aero,88194060,314.0,2.4178,10.7640,37.9745,14.664,33.6336
3,International,Paris Gare de Lyon,87686006,Geneve,85010082,503.0,1.7102,18.6645,59.0396,25.427,58.3198
4,TGV,Marne la Vallee Chessy,87111849,Avignon TGV,87318964,646.0,1.8734,24.1845,NaN,32.947,75.5678


On doit également filtrer les lignes qui sont des trains internationaux qui ne sont pas forcément sur le réseau français.

In [8]:
print(emissions["Transporteur"].unique())

['TGV' 'International' 'TER' 'Intercités']


In [9]:
def filter_transporteur(emissions_df):
    emissions_df = emissions_df.query("Transporteur != 'International'")
    return emissions_df

emissions = filter_transporteur(emissions)

In [10]:
color_palette = ["#af2bbf","#8770bf","#6c91bf","#5fb0b7","#5bc8af"]
#color_palette = ["#bbdef0","#00a6a6","#efca08","#f49f0a","#f08700"]


In [11]:
colors_moyens_de_transport = {"Train":color_palette[0], "Autocar":color_palette[1], "Avion":color_palette[2], "Voiture thermique":color_palette[3], "Voiture électrique":color_palette[4]}

In [12]:
def generate_line_chart(emissions_df, log_scale=False):
    fig = go.Figure()
    # On ajoute les courbes pour chaque moyen de transport
    emissions_df = emissions_df.sort_values(by='Distance')
    moyens_transport = ["Autocar", "Voiture électrique", "Voiture thermique", "Avion"]
    
    # Solutionn inspirée de https://github.com/plotly/plotly.py/issues/4278 pour afficher l'origine et la destination des trajets dans le hover
    # On ajoute une courbe invisible pour chaque moyen de transport, et on change son hovertext pour afficher l'origine et la destination
    fig.add_trace(go.Scatter(
        x=emissions_df['Distance'], 
        y=emissions_df['Train'], 
        opacity=0, 
        showlegend=False,
        hovertext=emissions_df['Origine'] + ' - ' + emissions_df['Destination'],
        hovertemplate='<b>%{hovertext}</b><extra></extra>'
        ))

    # On ajoute les courbes pour les trains, en montrant le transporteur dans le hover
    fig.add_trace(go.Scatter(
        x=emissions_df['Distance'],
        y=emissions_df['Train'],
        mode='lines+markers',
        name='Train',
        connectgaps=True,
        hovertext=emissions_df['Transporteur'],
        hovertemplate='<b>%{hovertext}</b> : %{y:.2f} kgCO2e <extra></extra>',
        marker=dict(color=colors_moyens_de_transport['Train'])
    ))
    for column in moyens_transport:
        fig.add_trace(go.Scatter(
            x=emissions_df['Distance'],
            y=emissions_df[column],
            mode='lines+markers',
            name=column,
            connectgaps=True,
            hovertemplate='%{y:.2f} kgCO2e',
            marker=dict(color=colors_moyens_de_transport[column])
        ))
    
    fig.update_layout(
        title='Émissions de CO2 par moyen de transport en fonction de la distance',
        xaxis_title='Distance entre les gares (km)',
        yaxis_title='CO2 émis en fonction de la distance',
        legend_title='Moyen de transport',
        hovermode='x unified'
    )
    
    if log_scale:
        fig.update_layout(title='Émissions de CO2 par moyen de transport en fonction de la distance (échelle logarithmique)')
        fig.update_yaxes(
            title='CO2 émis en fonction de la distance (échelle logarithmique)', 
            type="log"
    )
    
    return fig

generate_line_chart(emissions,False)    

In [ ]:
def generate_bar_chart(emissions_df):
    emissions_df_reduced = emissions_df[["Transporteur","Distance","Train","Autocar","Avion","Voiture électrique","Voiture thermique"]].copy()
    # On divise les émissions par la distance pour obtenir l'empreinte carbone par km
    emissions_df_reduced["Train"] = emissions_df_reduced["Train"] / emissions_df_reduced["Distance"]
    emissions_df_reduced["Autocar"] = emissions_df_reduced["Autocar"] / emissions_df_reduced["Distance"]
    emissions_df_reduced["Avion"] = emissions_df_reduced["Avion"] / emissions_df_reduced["Distance"]
    emissions_df_reduced["Voiture électrique"] = emissions_df_reduced["Voiture électrique"] / emissions_df_reduced["Distance"]
    emissions_df_reduced["Voiture thermique"] = emissions_df_reduced["Voiture thermique"] / emissions_df_reduced["Distance"]
    emissions_df_reduced = emissions_df_reduced.drop(columns=["Distance"])
    
    # On fait un melt pour avoir une colonne pour chaque moyen de transport. On a donc une dataframe longue avec une colonne 
    # pour le moyen de transport et une colonne pour l'empreinte carbone
    emissions_df_reduced = emissions_df_reduced.melt(id_vars=["Transporteur"], var_name="Moyen_de_transport", value_name="Empreinte carbone")
    
    emissions_df_reduced_train = emissions_df_reduced.query("Moyen_de_transport == 'Train'")
    emissions_df_reduced_train = emissions_df_reduced_train[["Transporteur","Empreinte carbone"]].groupby(["Transporteur"]).mean().reset_index()
    emissions_df_reduced_train = emissions_df_reduced_train.rename(columns={"Transporteur":"Moyen_de_transport"})
    
    emissions_df_reduced_other = emissions_df_reduced.query("Moyen_de_transport != 'Train'")
    emissions_df_reduced_other = emissions_df_reduced_other[["Moyen_de_transport","Empreinte carbone"]].groupby(["Moyen_de_transport"]).mean().reset_index()
    
    # On doit faire des subplots afin de pouvoir avoir des largeurs de colonnes différentes selon si c'est un train ou un autre moyen de transport
    fig = make_subplots(rows=1, cols=2, subplot_titles=["Train", "Autres moyens de transport"], shared_yaxes=True, column_widths=[1, 4], horizontal_spacing=0)
    fig.add_trace(go.Bar(
        x=emissions_df_reduced_train["Moyen_de_transport"],
        y=emissions_df_reduced_train["Empreinte carbone"],
        name='Train',
        marker_color=colors_moyens_de_transport['Train']
    ), row=1, col=1)
    
    for index, row in emissions_df_reduced_other.iterrows():
        fig.add_trace(go.Bar(
            x=[row["Moyen_de_transport"]],
            y=[row["Empreinte carbone"]],
            name=row["Moyen_de_transport"],
            marker_color=colors_moyens_de_transport[row["Moyen_de_transport"]]
        ), row=1, col=2)
        
    fig.update_layout(
        barcornerradius=15,
        yaxis_title='Empreinte carbone moyenne (kgCO2e/km)',
        legend_title='Moyen de transport',
        title='Empreinte carbone moyenne par km pour chaque moyen de transport',
    )
    
    return fig
generate_bar_chart(emissions)